# Class 11 - SQL cont'd, Web Applications

## How to update records in your tables

### CRUD operations

+ Create: `CREATE`
+ Retrieve `SELECT`
+ Update: `UPDATE`
+ Delete: `DELETE`

**SAMPLE STRUCTURE:** 
        
        `UPDATE table SET col1 = val1 col2 = val2 WHERE expression`

Other examples:

+ `UPDATE restaurant SET price = 16 WHERE price = 2;`
+ `UPDATE restaurant SET name = "Goofy Joe And The Sandwiches" WHERE id = 151;`

## Web Applications

### **What they can do:** 
+ Fetch data from a database and returning it to the web browser
+ Take in user data and manipulate it in some way
+ Other cool stuff! 

We want to understand the basics of web applications... but not become web developers! 

**Very rough cut of what web applications kinda do**: there's a computer program hanging out on a server and when a browser makes a request to the program, the server sends back the program's response! 

We're **NOT** writing the software that makes requests; we're writing the software that sends responses. 

Our server and browser need to use the same language: **HTTP (HyperText Transfer Protocol)**.  

### Writing a web request "by hand"

**When browsers send requests:**

+ Opens up a internet connection
+ Starts talking in HTTP!


+ Sample of what that looks like:

    + `nc www.columbia.edu 80`: net cat -- we usually use port 80; it'll look like nothing happened; this line connects to www.columbia.edu
    + `GET / HTTP/1.1` : this is an HTTP request; `GET` is the HTTP method; the `/` is the path to what we want to retrieve; `HTTP/1.1` is part of the htt protocol. 
    + `HOST: www.columbia.edu`: this is a header we need to pass through with our request;
    + The server responses with the "HTTP response" -- it includes a response code `200` when things have gone well. Another: `404` for not found. 
    + In the response, we got a lot of headers: date, server, content-type, etc. 
    + Actual content of the response -- the html at www.columbia.edu
    

There are the requests that drive the web! A **browser** is basically an elaborate software for writing and interpreting HTTP requests and responses (and then renders the html). 

Under **Developer Tools** on your brower, check out the **Network** tab; this tracks and shows you all the requests your browser is sending. 

### Our goal 
We need to write software that can parse HTTP requests and correctly format an HTTP response to send back. Built in library called "sockets" allows us to accept web connections in python. But that's a fair amount of work. We're going to use a **web framework** (a python library that has the software to parse HTTP requests for you). We would write a function in our code that is run whenever a web browser makes a request to our computer. 

Sample web frameworks: Django (python), node.js (javascript), Flask (python), Jekyll (ruby), Ruby on Rails

**Flask** is pretty nimble, so that's what we're going to use. 

## Writing web applications

In [1]:
!pip3 install Flask

    100% |████████████████████████████████| 81kB 3.1MB/s 
  Using cached Werkzeug-0.11.10-py2.py3-none-any.whl
  Using cached Jinja2-2.8-py2.py3-none-any.whl
    100% |████████████████████████████████| 286kB 2.2MB/s 
  Running setup.py bdist_wheel for click ... - \ done
  Stored in directory: /Users/rashida/Library/Caches/pip/wheels/b0/6d/8c/cf5ca1146e48bc7914748bfb1dbf3a40a440b8b4f4f0d952dd
Successfully built click


Usually, we'd write standalone python files (.py files). In class we wrote a very simple flask app in a standalone file called `Hello.py`:

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route("/hello")
def hello():
    return "Hello, world"

app.run()

In [ ]:
# Run our standalone file:
!python hello.py
# Once we run that program, nothing happens! It's literally just sitting there waiting on a browswer to make requests. 

Go to a web browser and put in this request: http://127.0.0.1:5000/hello. You should see "Hello, world" there. 

    

### Error messages you may have encountered:

+ Our application is set to only respond to requests made to a particular path! http://127.0.0.1:5000/hello. What will not work:  http://127.0.0.1:5000/ (because we didn't set it up to handle `/`, just `hello`. 
    + Flask has default errors when a browser tries to request a path it doesn't have something for. 
    
    
+ If you saw the "Address already in use" message, you probably run our sample Flask app code in Jupyter notebook AND Terminal/Babun. But we can only run it in one place. Can't start two applications that are servers on the same port at the same time! 
    + You could change the port by adjusting our code -- `app.run(port=5001)`
    + If you did start a Flask application from your notebook and you want to stop it, just interrupt the kernel (`Cmd-I`). As soon as it changes from an asterisk to a number, you'll know that it has stopped.
    + Use Ctrl-C to stop the application from Terminal (in the window where it is running). 

For the MOST part, you'd run applications from the command line. We're just going to run the Flask application in our notebooks -- only for the convenience of notes/teaching. **IN ACTUAL USAGE**, you wouldn't write the application in your notebook; you'd write a .py file that you could run from a server (just as we ran a scraper on our Digital Ocean droplets for Foundation). 

In [ ]:
# localhost/ the ip address below refers to your computer 
localhost = 127.0.0.1

In [ ]:
# curl goes to the URL, fetches the contents of that URL
# This will show us the results of our web request on the command line (rather than on the web browser)
!curl -s http://localhost:5000/hello

In [5]:
# Now our application has multiple paths/requests it can handle. It returns different strings depending on which 
# path you visit/request from. 

from flask import Flask
app = Flask(__name__) # `app` is object that represents our web application
# Python interprets the double underscore'd thing as the name of your program. 
# Evaluates to a unique value for your particular program.  

@app.route("/blah") # Whenever some accesses this path/this request is made, run the function that directly follows. 
def hello(): # This is the functionality of our program! This is the body of what it does. 
    return "Hello, world"

@app.route("/hello")
def fun_response_yay():
    return "you went to the path /hello! congratulations"

@app.route("/") # route resource or index resource
def welcome():
    return "Welcome to our Example Server!"

app.run() # causes your application to start listening for those HTTP requests. 

# The names of the function do not map to the paths -- that is arbitrary. 

The `@` is called a decorator -- a decorator is a function that takes another function as a parameter. We're just going to treat:
    
        @app.route("/blah")
        def hello():
            return "Hello, world"
            
as a set bit of function. 

**Flask applications: for every path, there's a function that should be executed for that path.** 


Aside: everything beyond the `?` in a URL is called the **query string**. It's a way of passing additional information with our HTTP request. **So how do we write a Flask application that responds different depending on what is passed in the query string?**

## The query string

We want to write an app that works like this: 

        request: curl -s http://localhost:5000/reverse?word=mammoth
        response: htommam
        
### The functionality of our program

In [7]:
''.join(reversed("mammoth"))

'htommam'

In [8]:
list(reversed("mammoth"))

['h', 't', 'o', 'm', 'm', 'a', 'm']

In [10]:
''.join(list(reversed("mammoth")))

'htommam'

In [11]:
''.join(reversed("mammoth"))

'htommam'

In [14]:
from flask import Flask, request 
# request is a special object that Flask library provides to you; allows you to access the query string

app = Flask(__name__) 

@app.route("/reverse") 
def reverser():
    
    # `request.args` acts like a dictionary; we can use this to get what has been passed through the query string
    word = request.args['word'] # returns whatever is in the key-value pair for `word`
    word_in_reverse = ''.join(reversed(word))
    return word_in_reverse

app.run() 

# Test whether this works by running `curl -s http://localhost:5000/reverse?word=dessert` in command line! 

Given this application, think about how an web API might work! Think about how we structured queries to APIs like NYTimes and Spotify -- we passed a URL and received a response (not unlike what we're doing here). 

### What if we don't pass it a query string? 

Sample request: `curl -s http://localhost:5000/reverse`

**This throws us an error:** Bad Request. The browser (or proxy) sent a request that this server could not understand.

We'll need to adjust our application so that it can account for such requests. 

### `.get()` behavior

In [15]:
x = {'oranges': 6, 'apples': 4, 'durians': 2}

In [16]:
if 'passionfruit' in x:
    print(x['passionfruit'])

In [19]:
# Get the value stored for a key in a dictionary, and if that key does not exist, return a default value of 0
x.get('oranges', 0)

6

In [17]:
# Get the value stored for a key in a dictionary, and if that key does not exist, return a default value of 0
x.get('passionfruit', 0)

0

In [18]:
from flask import Flask, request 

app = Flask(__name__) 

@app.route("/reverse") 
def reverser():
    
    word = request.args.get('word', None) # will get value associated with "word" or else, return None (our default)
    if word: 
        word_in_reverse = ''.join(reversed(word))
        return word_in_reverse
    
    else:
        return "you did not use the API correctly. happens to us all."

app.run()

### A more interesting hello world

    request: http://localhost:5000/greeting?to_greet=galaxy
    reponse: Hello, galaxy! 
        
additionally: choose a random greeting.

    ["hello", "hi", "salutations", "howdy", "'sup", "hey"]

In [20]:
import random

In [21]:
# This is like rolling a 6-sided die
random.randrange(1,7)

6

In [22]:
# will produce random floating point data
random.uniform(0,1)

0.020730306387093944

In [23]:
random.choice(["hello", "hi", "salutations", "howdy", "'sup", "hey"])

'hey'

In [26]:
import random 
from flask import Flask, request 

app = Flask(__name__) 

# we can do our usual python code stuff here! 
# just as below! 

greets = ["hello", "hi", "salutations", "howdy", "'sup", "hey"]
punc = ["!", "!!", "?", ".", "..."]

@app.route("/greeting") 
def greet_generator():
    
    thing = request.args['to_greet']
    
    greeting = random.choice(greets) + " " + thing + random.choice(punc)
    return greeting
    

app.run()

### Adding HTML formatting to our response

In [29]:
import random 
from flask import Flask, request 

app = Flask(__name__) 

# we can do our usual python code stuff here! 
# just as below! 

greets = ["hello", "hi", "salutations", "howdy", "'sup", "hey"]
punc = ["!", "!!", "?", ".", "..."]

@app.route("/greeting") 
def greet_generator():
    
    thing = request.args['to_greet']
    
    greeting = random.choice(greets) + " " + thing + random.choice(punc)
    return "<h1>WELCOME TO GREET-O-TRON</h1>" + "<tt>" + greeting + "</tt>"
    

app.run()

Not all web applications return html. For example, the NYTimes API returns a json object. You can return different things! 

**On Thursday, we'll talk about how to use HTML to format responses and how to use HTML to have user submit a form (rather than editing the query string). We'll also add a databases component to our web applications.**